In [1]:
PROMPT_TEMPLATE = """
Ответь на вопрос, основываясь исключительно на предоставленном контексте:

{context}

---

Вопрос к данному контексту: {question}

---

Приоритетный язык ответа - русский
"""

PROMPT_TEMPLATE_SIMILARITY = """
Ответь на вопросы, основываясь исключительно на предоставленном контексте:

{context}

---

Вопросы к данному контексту:
{questions}

---

Приоритетный язык ответа - русский. отвечай на каждый вопрос в новой строке
"""

###Записала ключ в переменное окружение

In [2]:
!echo GROQ_API_KEY=\"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\" > .env

In [2]:
!more .env

GROQ_API_KEY="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"


# Ollama

In [4]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  38524      0 --:--:-- --:--:-- --:--:-- 38557
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [52]:
# run ollama
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [51]:
# stop ollama
!pkill ollama

In [20]:
%%capture
!ollama pull llama3
!ollama pull phi3
!ollama pull openhermes

!ollama pull snowflake-arctic-embed:137m
!ollama pull nomic-embed-text
!ollama pull mxbai-embed-large

In [48]:
!ollama list

NAME                       	ID          	SIZE  	MODIFIED          
mxbai-embed-large:latest   	468836162de7	669 MB	58 minutes ago   	
nomic-embed-text:latest    	0a109f422b47	274 MB	58 minutes ago   	
openhermes:latest          	95477a2659b7	4.1 GB	58 minutes ago   	
snowflake-arctic-embed:137m	12616299a158	274 MB	58 minutes ago   	
phi3:latest                	4f2222927938	2.2 GB	About an hour ago	
llama3:latest              	365c0bd3c000	4.7 GB	About an hour ago	


# Gradio

### Libraries

In [8]:
%%capture
!pip install gradio

!pip install chromadb
!pip install python-docx python-dotenv python-pptx
!pip install bs4

!pip install unstructured
!pip install langchain_community langchain_core langchain_text_splitters
!pip install langchain_chroma langchain_openai

!pip install groq

!pip install sentence-transformers

### Imports

In [5]:
import os
import shutil
import dotenv

import gradio as gr

# from langchain_community.vectorstores import Chroma, FAISS
from langchain_chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_community.llms.ollama import Ollama

from langchain_community.document_loaders import BSHTMLLoader, CSVLoader, JSONLoader, TextLoader
from langchain_community.document_loaders.word_document import UnstructuredWordDocumentLoader
from langchain_community.document_loaders.excel import UnstructuredExcelLoader
from langchain_community.document_loaders.powerpoint import UnstructuredPowerPointLoader
from langchain_community.document_loaders.pdf import PyPDFLoader

from langchain_core.documents import Document

from langchain_text_splitters import RecursiveCharacterTextSplitter

from groq import Groq


import pandas as pd

from sentence_transformers import SentenceTransformer

In [6]:
dotenv.load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

## Main code

### Prepare Data

In [7]:
def load_documents(files: list[str]):
    documents = []
    for file in files:
        extension = file.split('.')[-1]
        if extension == 'txt':
            documents += TextLoader(file).load()
        if extension == 'csv':
            documents += CSVLoader(file).load()
        if extension == 'html':
            documents += BSHTMLLoader(file, open_encoding='utf-8').load()
        if extension == 'json':
            continue
            documents += JSONLoader(file, jq_schema='.quiz').load()
        if extension == 'pdf':
            documents += PyPDFLoader(file).load()
        if extension == 'xlsx':
            documents += UnstructuredExcelLoader(file).load()
        if extension == 'docx':
            documents += UnstructuredWordDocumentLoader(file).load()
        if extension == 'pptx':
            documents += UnstructuredPowerPointLoader(file).load()
    return documents


In [8]:
def split_documents(documents: list[Document], splitter):
    splitter_chunk_size = int(splitter.split(',')[0].split('=')[1].strip())
    splitter_chunk_overlap = int(splitter.split(',')[1].split('=')[1].strip())
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=splitter_chunk_size,
        chunk_overlap=splitter_chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    print("[INFO] split_documents")
    print("[DEBUG]", documents)
    return text_splitter.split_documents(documents)

In [9]:
def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}"
        if page:
            current_page_id += f":{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id
    print("[INFO] calculate_chunk_ids")
    return chunks

### Interface

In [53]:
def process_input(model_name, embeddings, splitter, files, question_file):
    documents = load_documents(files)


    chunks = split_documents(documents, splitter)
    chunks = calculate_chunk_ids(chunks)
    db = Chroma.from_documents(chunks, OllamaEmbeddings(model=embeddings))

    df=pd.read_csv(question_file, delimiter=';',encoding='utf-8', encoding_errors='ignore', header=None)

    questions = df[0][:-2].tolist()
    answers = df[1][:-2].tolist()
    print(questions)


    headers=['Construction','questions','right_answer','predict_answer','Semantic Textual Similarity']

    resultdf = pd.DataFrame(columns=headers)
    counter=0
    for i, question in enumerate(questions):
        results = db.similarity_search_with_score(question, k=5)
        context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
        prompt = PROMPT_TEMPLATE.format(context=context_text, question=question)
        if model_name == "llama3.1":
            client = Groq(
                api_key=GROQ_API_KEY,
            )

            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model="llama3-70b-8192",
            )
            resultdf.loc[i]=[f'{model_name}-{embeddings}-{splitter}',question,answers[i],chat_completion.choices[0].message.content,None]

        else:
            model = Ollama(model=model_name)
            resultdf.loc[i]=[f'{model_name}-{embeddings}-{splitter}', question,answers[i],model.invoke(prompt),None]

    simModel = SentenceTransformer("all-MiniLM-L6-v2")
    for i in range(resultdf.shape[0]):
      embeddings1 = simModel.encode(resultdf['right_answer'][i])
      embeddings2 = simModel.encode(resultdf['predict_answer'][i])
      resultdf['Semantic Textual Similarity'][i] = simModel.similarity(embeddings1, embeddings2)[0][0]
      if resultdf['Semantic Textual Similarity'][i]>=0.6:
        counter+=1

    #resultdf.to_csv(f'{model}-{embeddings}-{splitter}.csv', sep=';', encoding='utf-8', index=False)
    return counter


In [55]:
models = ["llama3", "phi3", "openhermes", "llama3.1"]
embeddings = ["nomic-embed-text", "snowflake-arctic-embed:137m", "mxbai-embed-large"]
splitters = ["chunk_size=1000, chunk_overlap=250", "chunk_size=500, chunk_overlap=50", "chunk_size=300, chunk_overlap=0"]

results=[]

counter=0
for model in models:
  for embedding in embeddings:
    for splitter in splitters:
      counter+=1
      print(counter)
      try:
        res=process_input(model, embedding, splitter, ['Каракал.pdf'], 'Каракал.csv')
        print(f'{model}-{embedding}-{splitter}')
        print(res)
        print()
        results.append([f'{model}-{embedding}-{splitter}',res])
      except:
        print(f'error {model}-{embedding}-{splitter}')
        continue


# a=process_input('llama3.1',
#               "nomic-embed-text",
#               'chunk_size=1000, chunk_overlap=250',
#               ['Каракал.pdf'],'Каракал.csv')
# print(a)

1
[INFO] split_documents
[DEBUG] [Document(metadata={'source': 'Каракал.pdf', 'page': 0}, page_content='     Каракал  \nОписание вида  \nПо меркам дикой природы каракалы (от тюркского qara qulaq — «черное ухо») — \nкошки средних размеров, но на контрасте с обычными домашними питомцами они \nвыглядят внушительно. Это высокие и стройные хищники с длинными \nконечностями, шерстью песочного или красн о-бурого цвета, большими ушами с \nчерной окраской и характерными кисточками волос на концах.  \nНесмотря на «дикое» происхождение, таких кошек довольно легко приручить. \nЕсли их правильно воспитывать и обращаться с ними доброжелательно, они не \nпроявляют агресси и по отношению к людям. И все же авторитет в глазах каракала \nнужно уметь заслужить. Новоиспеченному владельцу важно стать для котенка \nнепререкаемым авторитетом. В противном случае питомец будет стремиться \nдоминировать над окружающими, и с его поведением мо гут возникнуть большие \nпроблемы. Так что людям без опыта обращения с 

In [43]:
# тут у меня была идея смотреть качество модели по суммарным значениям тензора, но я от неё отказалась, ибо,
# тогда, если на каждый вопрос, модель даст стабильно 0.4(плохой) ответ,
# то суммарный тензор будет лучше, чем, когда модель даст максимально правильыне ответы хотя бы на половину.

models = ["llama3","llama3.1", "phi3", "openhermes"]
embeddings = ["nomic-embed-text", "snowflake-arctic-embed:137m", "mxbai-embed-large"]
splitters = ["chunk_size=500, chunk_overlap=50","chunk_size=1000, chunk_overlap=250" , "chunk_size=300, chunk_overlap=0"]

results=[]

counter=0
for splitter in splitters:

  for embedding in embeddings:
    for model in models:
      counter+=1
      print(counter)
      res=process_input(model, embedding, splitter, ['Каракал.pdf'], 'Каракал.csv')
      print(f'{model}-{embedding}-{splitter}')
      print(res)
      print()
      results.append([f'{model}-{embedding}-{splitter}',res])


# a=process_input('llama3.1',
#               "nomic-embed-text",
#               'chunk_size=1000, chunk_overlap=250',
#               ['Каракал.pdf'],'Каракал.csv')
# print(a)

1
[INFO] split_documents
[DEBUG] [Document(metadata={'source': 'Каракал.pdf', 'page': 0}, page_content='     Каракал  \nОписание вида  \nПо меркам дикой природы каракалы (от тюркского qara qulaq — «черное ухо») — \nкошки средних размеров, но на контрасте с обычными домашними питомцами они \nвыглядят внушительно. Это высокие и стройные хищники с длинными \nконечностями, шерстью песочного или красн о-бурого цвета, большими ушами с \nчерной окраской и характерными кисточками волос на концах.  \nНесмотря на «дикое» происхождение, таких кошек довольно легко приручить. \nЕсли их правильно воспитывать и обращаться с ними доброжелательно, они не \nпроявляют агресси и по отношению к людям. И все же авторитет в глазах каракала \nнужно уметь заслужить. Новоиспеченному владельцу важно стать для котенка \nнепререкаемым авторитетом. В противном случае питомец будет стремиться \nдоминировать над окружающими, и с его поведением мо гут возникнуть большие \nпроблемы. Так что людям без опыта обращения с 

KeyboardInterrupt: 

In [ ]:
iface = gr.Interface(fn=process_input,
                     inputs=[
                         gr.Dropdown(label="Model",
                                     choices=[
                                         "llama3",
                                         "phi3",
                                         "openhermes",
                                         "llama3.1"
                                         ],
                                     value="llama3"),
                         gr.Dropdown(label="Embedding",
                                     choices=[
                                         "nomic-embed-text",
                                         "snowflake-arctic-embed:137m",
                                         "mxbai-embed-large"
                                         ],
                                     value="nomic-embed-text"),
                         gr.Dropdown(label="Splitter",
                                     choices=["chunk_size=1000, chunk_overlap=250",
                                              "chunk_size=500, chunk_overlap=50",
                                              "chunk_size=300, chunk_overlap=0"],
                                     value="chunk_size=1000, chunk_overlap=250"),
                         gr.Files(file_types=["pdf", "txt", "pptx", "csv", "html", "xlsx", "docx"],
                                  label="Load files to RAG"),
                         gr.File(file_types=["csv"],
                                 label="Load file with questions to RAG"),
                         gr.Textbox(label="Question")],
                     outputs="text",
                     title="Document Query with Ollama",
                     description="Enter files and a question to query the documents.",
                     allow_flagging='auto')

In [ ]:
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1fbe86aa67ae3307a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[INFO] split_documents
[DEBUG] [Document(metadata={'source': '/tmp/gradio/86dfe6eed131ca9b5cf45709c932b45cdfcce42595fefd7688ec9711efff1c4e/Каракал.pdf', 'page': 0}, page_content='     Каракал  \nОписание вида  \nПо меркам дикой природы каракалы (от тюркского qara qulaq — «черное ухо») — \nкошки средних размеров, но на контрасте с обычными домашними питомцами они \nвыглядят внушительно. Это высокие и стройные хищники с длинными \nконечностями, шерстью песочного или красн о-бурого цвета, большими ушами с \nчерной окраской и характерными кисточками волос на концах.  \nНесмотря на «дикое» происхождение, таких кошек довольно легко приручить. \nЕсли их правильно воспитывать и обращаться с ними доброжелательно, они не \nпроявляют агресси и по отношению к людям. И все же авторитет в глазах каракала \nнужно уметь заслужить. Новоиспеченному владельцу важно стать для котенка \nнепререкаемым авторитетом. В противном случае питомец будет стремиться \nдоминировать над окружающими, и с его поведением

In [ ]:
import csv
import pandas as pd


df=pd.read_csv('Каракал.csv',delimiter=';',encoding='utf-8', encoding_errors='ignore',header=None)

questions = df[0][:-2].tolist()
print(questions)


In [ ]:
df=pd.read_csv("Каракал.csv", delimiter=';',encoding='utf-8', encoding_errors='ignore', header=None)

questions = df[1][:-2].tolist()
print(questions)



In [ ]:
df2=pd.read_csv("flagged/log.csv", encoding='utf-8', encoding_errors='ignore')

questions2=df2['output'].tolist()
print(questions2)

In [ ]:
result = []
for i in range(len(questions2)):
  embeddings1 = model.encode(questions[i])
  embeddings2 = model.encode(questions2[i])
  result.append((questions[i], questions2[i], model.similarity(embeddings1, embeddings2)[0][0]))

In [ ]:
for res in result:
  print(res)